In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import pandas as pd
import numpy as np
import cv2

### 15 đặc trưng

In [ ]:
data={
    'Rank':[],
    'Year':[],
    'Movie':[],
    'WorldwideBox Office':[], # doanh thu toàn cầu
    'Production Budget':[], # vốn đầu tư
    'Date Releases':[], # ngày phát hành
    'MPAA':[], # nhãn phim(thuộc thể loại nào, độ tuổi nào,...)
    'Running Time':[],
    'Franchise':[], # tên thương mại
    'Genre':[], # thể loại
    'Creative Type':[], # loại quảng cáo
    'Production/Financing Companies':[], # công ty sản xuất
    'Production Countries':[], # quốc gia phát hành
    'Languages':[], # ngôn ngữ
    'Director':[] # đạo diễn
}
headers={
'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.64'
}


for page in range(1,100,100):
    url='https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/'+str(page)
    response=requests.get(url,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    table_html=soup.find('table')

    for tr in table_html.find_all('tr')[1:11]:

        for key in data:
            data[key].append('')

        if(tr.find_all('td')):
            data['Rank'][-1]=(tr.find_all('td')[0].text)
            data['Year'][-1]=(tr.find_all('td')[1].text)
            data['WorldwideBox Office'][-1]=(tr.find_all('td')[3].text)

        all_a_tags=tr.find_all('a')
        if all_a_tags:
            link_movie=all_a_tags[-1].get('href').split('/')[2].split('#')[0]
            data['Movie'][-1]=link_movie
            
            url_summary='https://www.the-numbers.com/movie/' + link_movie +'#tab=summary'
            response_summary=requests.get(url_summary,headers=headers)
            soup_summary=BeautifulSoup(response_summary.content,'html.parser')
            summary=soup_summary.find(id='summary')

            for title in summary.find_all('h2'):
                if(title.text=='Metrics'):
                    metrics_table=title.next_sibling
                    for tr_metrics_table in metrics_table.find_all('tr'):
                        if(tr_metrics_table.find_all('td')):
                            th=tr_metrics_table.find_all('td')[0]
                            td=tr_metrics_table.find_all('td')[-1]
                            if(th.text=='Production\xa0Budget:'):
                                data['Production Budget'][-1]=td.text.split(' ')[0]
                                break

                if(title.text=='Movie Details'):
                    detail_table=title.next_sibling.next_sibling
                    for tr_detail_table in detail_table.find_all('tr'):
                        if(tr_detail_table.find_all('td')):
                            th=tr_detail_table.find_all('td')[0]
                            td=tr_detail_table.find_all('td')[-1]

                            if(th.text=='Running Time:'):
                                data['Running Time'][-1]=(td.text )
                            if(th.text=='Genre:'):
                                data['Genre'][-1]=(td.text )
                            if(th.text=='Creative\xa0Type:'):
                                data['Creative Type'][-1]=(td.text )
                            if(th.text=='Production/Financing Companies:'):
                                data['Production/Financing Companies'][-1]=(td.text )
                            if(th.text=='Production Countries:'):
                                data['Production Countries'][-1]=(td.text )
                            if(th.text=='Languages:'):
                                data['Languages'][-1]=(td.text )
                            if(th.text=='International Releases:'):
                                data['Date Releases'][-1]=(td.text.split('(')[0] )
                            if(th.text=='MPAA\xa0Rating:'):
                                data['MPAA'][-1]=(td.text.split(' ')[0] )
                            if(th.text=='Franchise:'):
                                data['Franchise'][-1]=(td.text )


                    break

            url_cast='https://www.the-numbers.com/movie/' + link_movie +'#tab=cast-and-crew'
            response_cast=requests.get(url_cast,headers=headers)
            soup_cast=BeautifulSoup(response_cast.content,'html.parser')
            cast=soup_cast.find(id='cast-and-crew')
            for title in cast.find_all('h1'):
                if(title.text=='Production and Technical Credits'):
                    cast_table=title.next_sibling.next_sibling
                    director_list=[]
                    for cast_td in cast_table.find_all('td'):
                        if(cast_td.text=='Director'):
                            director_list.append(cast_td.previous_sibling.previous_sibling.previous_sibling.previous_sibling.text)
                            data['Director'][-1]=','.join(director_list)
            


In [ ]:

df=pd.DataFrame(data)
df.to_csv('test_crawling.csv')
